In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import warnings
from pathlib import Path

# Configurações de visualização
plt.style.use('ggplot')
pd.set_option('display.max_columns', 50)
warnings.filterwarnings('ignore')

# Modelo de Previsão de Emissões de N2O no Brasil

Este notebook apresenta um modelo otimizado para previsão de emissões de óxido nitroso (N2O) no Brasil,
com foco em superar o baseline simplista fornecido pelo professor.

## 1. Compreensão e Preparação dos Dados

In [ ]:
def load_data(file_path):
    """
    Carrega os dados do arquivo CSV e retorna um DataFrame.
    
    Args:
        file_path: Caminho para o arquivo CSV.
        
    Returns:
        DataFrame pandas com os dados carregados.
    """
    print(f"Carregando dados de {file_path}...")
    return pd.read_csv(file_path, encoding='utf-8')


def explore_data(data):
    """
    Explora os dados e exibe estatísticas básicas.
    
    Args:
        data: DataFrame pandas para exploração.
    """
    print("\n=== EXPLORAÇÃO INICIAL DOS DADOS ===")
    
    # Verificar as primeiras linhas do dataset
    print("\nPrimeiras 5 linhas do dataset:")
    print(data.head())
    
    # Informações das colunas
    print("\nColunas disponíveis:", data.columns.tolist())
    print(f"Número total de registros: {len(data)}")
    
    # Resumo estatístico
    print("\nInformações gerais do dataset:")
    print(data.info())
    
    # Resumo estatístico das colunas numéricas
    print("\nResumo estatístico das colunas numéricas:")
    print(data.describe())
    
    # Verificar valores nulos
    missing_values = data.isnull().sum()
    print("\nValores nulos por coluna:")
    print(missing_values)
    
    # Porcentagem de valores nulos
    missing_percentage = (missing_values / len(data)) * 100
    print("\nPorcentagem de valores nulos por coluna:")
    for col, pct in missing_percentage.items():
        if pct > 0:
            print(f"{col}: {pct:.2f}%")


def filter_gas_data(data, gas_name):
    """
    Filtra os dados para um gás específico e exibe informações sobre o subconjunto.
    
    Args:
        data: DataFrame pandas com todos os dados.
        gas_name: Nome do gás para filtragem.
        
    Returns:
        DataFrame pandas contendo apenas os dados do gás especificado.
    """
    print(f"\n=== FILTRANDO DADOS DE {gas_name} ===")
    filtered_data = data[data['gas'].str.contains(gas_name, case=False, na=False)]
    
    print(f"Registros de {gas_name}: {len(filtered_data)}")
    print("\nResumo estatístico:")
    print(filtered_data.describe())
    
    return filtered_data


# Carregar dados
data_path = "../data/br_seeg_emissoes_brasil.csv"
data = load_data(data_path)

# Explorar dados
explore_data(data)

# Filtrar dados de N2O
n2o_data = filter_gas_data(data, 'N2O')

## 2. Análise Exploratória (EDA)

In [ ]:
def plot_emissions_by_year(data, gas_name):
    """
    Gera um gráfico de linha mostrando emissões por ano.
    
    Args:
        data: DataFrame com os dados filtrados.
        gas_name: Nome do gás para o título.
    """
    emissions_by_year = data.groupby('ano')['emissao'].sum().reset_index()
    
    plt.figure(figsize=(12, 6))
    plt.plot(emissions_by_year['ano'], emissions_by_year['emissao'], marker='o', linewidth=2)
    plt.title(f'Emissões Totais de {gas_name} por Ano', fontsize=14)
    plt.xlabel('Ano', fontsize=12)
    plt.ylabel('Emissão Total (t)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    
    # Adicionar anotações para eventos importantes
    plt.annotate('Salto em 1990', xy=(1990, 500000), xytext=(1987, 300000),
                arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8))
    
    plt.tight_layout()
    plt.show()


def plot_emissions_by_sector(data, gas_name):
    """
    Gera um gráfico de barras mostrando emissões por setor.
    
    Args:
        data: DataFrame com os dados filtrados.
        gas_name: Nome do gás para o título.
    """
    emissions_by_sector = data.groupby('nivel_1')['emissao'].sum().sort_values(ascending=False).reset_index()
    
    plt.figure(figsize=(12, 6))
    bar_plot = sns.barplot(data=emissions_by_sector, x='nivel_1', y='emissao')
    
    # Adicionar rótulos com os valores percentuais
    total = emissions_by_sector['emissao'].sum()
    for i, p in enumerate(bar_plot.patches):
        percentage = 100 * p.get_height() / total
        bar_plot.annotate(f'{percentage:.1f}%',
                          (p.get_x() + p.get_width() / 2., p.get_height()),
                          ha='center', va='bottom', fontsize=10)
    
    plt.title(f'Emissões de {gas_name} por Setor', fontsize=14)
    plt.xlabel('Setor', fontsize=12)
    plt.ylabel('Emissão Total (t)', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    plt.show()


def plot_emission_distribution(data, gas_name):
    """
    Gera gráficos para visualizar a distribuição das emissões.
    
    Args:
        data: DataFrame com os dados filtrados.
        gas_name: Nome do gás para o título.
    """
    plt.figure(figsize=(14, 10))
    
    # Histograma da distribuição original (log scale)
    plt.subplot(2, 2, 1)
    plt.hist(data['emissao'][data['emissao'] > 0], bins=50, log=True)
    plt.title(f'Distribuição das Emissões de {gas_name} (Log Scale)', fontsize=12)
    plt.xlabel('Emissão (t)', fontsize=10)
    plt.ylabel('Frequência (log)', fontsize=10)
    
    # Boxplot por nível hierárquico 1
    plt.subplot(2, 2, 2)
    sns.boxplot(data=data, x='nivel_1', y='emissao')
    plt.title(f'Boxplot de Emissões de {gas_name} por Setor', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yscale('log')
    
    # Boxplot por ano (agrupado)
    years_to_show = list(range(1970, 2020, 10))
    plt.subplot(2, 2, 3)
    year_data = data[data['ano'].isin(years_to_show)]
    sns.boxplot(data=year_data, x='ano', y='emissao')
    plt.title(f'Boxplot de Emissões de {gas_name} por Década', fontsize=12)
    plt.yscale('log')
    
    # Scatterplot ano vs emissão (com alta transparência)
    plt.subplot(2, 2, 4)
    plt.scatter(data['ano'], data['emissao'], alpha=0.1, s=10)
    plt.title(f'Dispersão de Emissões de {gas_name} por Ano', fontsize=12)
    plt.xlabel('Ano', fontsize=10)
    plt.ylabel('Emissão (t)', fontsize=10)
    plt.yscale('log')
    
    plt.tight_layout()
    plt.show()


# Plotar análises
plot_emissions_by_year(n2o_data, 'N2O')
plot_emissions_by_sector(n2o_data, 'N2O')
plot_emission_distribution(n2o_data, 'N2O')

## 3. Tratamento de Dados e Engenharia de Features

In [ ]:
def handle_missing_values(data):
    """
    Trata valores ausentes nos dados.
    
    Args:
        data: DataFrame a ser processado.
        
    Returns:
        DataFrame com valores ausentes tratados.
    """
    print("\n=== TRATAMENTO DE VALORES AUSENTES ===")
    
    # Copiar dados para não modificar o original
    processed_data = data.copy()
    
    # Verificar valores nulos originais
    null_before = processed_data.isnull().sum()
    null_pct_before = (null_before / len(processed_data)) * 100
    
    # Preencher valores ausentes em colunas categóricas com "Desconhecido"
    cat_columns = processed_data.select_dtypes(include=['object']).columns
    for col in cat_columns:
        if null_before[col] > 0:
            processed_data[col].fillna('Desconhecido', inplace=True)
    
    # Preencher valores ausentes em 'emissao' com 0
    if 'emissao' in processed_data.columns and null_before['emissao'] > 0:
        processed_data['emissao'].fillna(0, inplace=True)
    
    # Verificar valores nulos após processamento
    null_after = processed_data.isnull().sum()
    
    # Mostrar resumo do tratamento
    print("Colunas tratadas para valores ausentes:")
    for col in null_before.index:
        if null_before[col] > 0:
            print(f"  {col}: {null_before[col]} valores ausentes ({null_pct_before[col]:.2f}%) → {null_after[col]} restantes")
    
    return processed_data


def handle_outliers(data, column, method='iqr', threshold=1.5):
    """
    Detecta e trata outliers em uma coluna específica.
    
    Args:
        data: DataFrame contendo os dados.
        column: Nome da coluna a ser tratada.
        method: Método para detecção de outliers ('iqr' ou 'zscore').
        threshold: Limite para considerar um valor como outlier.
        
    Returns:
        DataFrame com outliers tratados.
    """
    print(f"\n=== TRATAMENTO DE OUTLIERS NA COLUNA '{column}' ===")
    
    # Copiar dados para não modificar o original
    processed_data = data.copy()
    original_count = len(processed_data)
    
    if method.lower() == 'iqr':
        # Método IQR (Interquartile Range)
        Q1 = processed_data[column].quantile(0.25)
        Q3 = processed_data[column].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        
        print(f"Limites calculados pelo método IQR:")
        print(f"  Q1 = {Q1:.2f}")
        print(f"  Q3 = {Q3:.2f}")
        print(f"  IQR = {IQR:.2f}")
        print(f"  Limite inferior = {lower_bound:.2f}")
        print(f"  Limite superior = {upper_bound:.2f}")
        
        # Identificar outliers
        outliers = processed_data[(processed_data[column] < lower_bound) | 
                                 (processed_data[column] > upper_bound)]
        
        # Tratar outliers (substituir por limites)
        processed_data.loc[processed_data[column] > upper_bound, column] = upper_bound
        processed_data.loc[processed_data[column] < lower_bound, column] = lower_bound
        
    elif method.lower() == 'zscore':
        # Método Z-score
        from scipy import stats
        z_scores = stats.zscore(processed_data[column])
        abs_z_scores = np.abs(z_scores)
        outliers = processed_data[abs_z_scores > threshold]
        
        # Tratar outliers (substituir por limite do z-score)
        mean_val = processed_data[column].mean()
        std_val = processed_data[column].std()
        upper_bound = mean_val + threshold * std_val
        lower_bound = mean_val - threshold * std_val
        
        print(f"Limites calculados pelo método Z-score:")
        print(f"  Média = {mean_val:.2f}")
        print(f"  Desvio padrão = {std_val:.2f}")
        print(f"  Limite inferior = {lower_bound:.2f}")
        print(f"  Limite superior = {upper_bound:.2f}")
        
        processed_data.loc[processed_data[column] > upper_bound, column] = upper_bound
        processed_data.loc[processed_data[column] < lower_bound, column] = lower_bound
    
    # Resumo do tratamento
    print(f"Outliers detectados: {len(outliers)} ({len(outliers)/original_count*100:.2f}% dos dados)")
    print(f"  Média antes do tratamento: {data[column].mean():.2f}")
    print(f"  Média após tratamento: {processed_data[column].mean():.2f}")
    print(f"  Mediana antes: {data[column].median():.2f}")
    print(f"  Mediana após: {processed_data[column].median():.2f}")
    
    return processed_data


def create_features(data):
    """
    Cria novas features a partir das existentes.
    
    Args:
        data: DataFrame com os dados originais.
        
    Returns:
        DataFrame com as novas features adicionadas.
    """
    print("\n=== ENGENHARIA DE FEATURES ===")
    
    # Copiar dados para não modificar o original
    enhanced_data = data.copy()
    
    # 1. Features temporais: década
    enhanced_data['decada'] = (enhanced_data['ano'] // 10) * 10
    
    # 2. Feature para identificar se a emissão está relacionada à agropecuária
    enhanced_data['is_agropecuaria'] = (enhanced_data['nivel_1'] == 'Agropecuária').astype(int)
    
    # 3. Features para combinações importantes de níveis hierárquicos
    enhanced_data['nivel_1_2'] = enhanced_data['nivel_1'] + '_' + enhanced_data['nivel_2']
    
    # 4. Feature para identificar tipo de emissão (simplificação)
    if 'tipo_emissao' in enhanced_data.columns:
        emission_type_map = {'Emissão': 1, 'Remoção': -1}
        enhanced_data['emission_factor'] = enhanced_data['tipo_emissao'].map(emission_type_map).fillna(0)
    
    # 5. Transformação logarítmica para valores positivos de emissão (ajuda a lidar com dados enviesados)
    enhanced_data['log_emissao'] = np.log1p(enhanced_data['emissao'].clip(lower=0))
    
    # Resumo das novas features
    print("Novas features criadas:")
    new_features = ['decada', 'is_agropecuaria', 'nivel_1_2', 'emission_factor', 'log_emissao']
    for col in new_features:
        if col in enhanced_data.columns:
            print(f"  - {col}")
    
    print(f"\nDimensões dos dados: {enhanced_data.shape[0]} linhas x {enhanced_data.shape[1]} colunas")
    
    return enhanced_data


# Tratar valores ausentes
n2o_data_processed = handle_missing_values(n2o_data)

# Tratar outliers na coluna de emissão
n2o_data_processed = handle_outliers(n2o_data_processed, 'emissao', method='iqr', threshold=3.0)

# Criar novas features
n2o_data_enhanced = create_features(n2o_data_processed)

## 4. Preparação para Modelagem

In [ ]:
def prepare_data_for_modeling(data, features, target, train_years, test_years):
    """
    Prepara os dados para modelagem, incluindo one-hot encoding e divisão de treino/teste.
    
    Args:
        data: DataFrame com os dados filtrados.
        features: Lista de colunas a serem usadas como features.
        target: Nome da coluna alvo.
        train_years: Range de anos para o conjunto de treino.
        test_years: Range de anos para o conjunto de teste.
        
    Returns:
        X_train, y_train, X_test, y_test para modelagem.
    """
    print("\n=== PREPARAÇÃO PARA MODELAGEM ===")
    print(f"Features selecionadas: {features}")
    print(f"Target: {target}")
    print(f"Anos de treino: {min(train_years)}-{max(train_years)}")
    print(f"Anos de teste: {min(test_years)}-{max(test_years)}")
    
    # One-hot encoding para variáveis categóricas
    categorical_features = [col for col in features if col != 'ano' and data[col].dtype == 'object']
    print(f"Features categóricas a serem codificadas: {categorical_features}")
    
    data_encoded = pd.get_dummies(data[features + [target]],
                                  columns=categorical_features)
    
    print(f"Dimensões após one-hot encoding: {data_encoded.shape[0]} linhas x {data_encoded.shape[1]} colunas")
    
    # Divisão temporal
    X_train = data_encoded[data_encoded['ano'].isin(train_years)].drop(columns=[target])
    y_train = data_encoded[data_encoded['ano'].isin(train_years)][target]
    X_test = data_encoded[data_encoded['ano'].isin(test_years)].drop(columns=[target])
    y_test = data_encoded[data_encoded['ano'].isin(test_years)][target]
    
    print(f"Conjuntos de dados:")
    print(f"  X_train: {X_train.shape[0]} linhas x {X_train.shape[1]} colunas")
    print(f"  y_train: {y_train.shape[0]} valores")
    print(f"  X_test: {X_test.shape[0]} linhas x {X_test.shape[1]} colunas")
    print(f"  y_test: {y_test.shape[0]} valores")
    
    return X_train, y_train, X_test, y_test


def impute_missing_values(X_train, X_test, y_train, y_test):
    """
    Imputa valores ausentes nos dados de treino e teste.
    
    Args:
        X_train, X_test, y_train, y_test: Dados para imputação.
        
    Returns:
        Dados com valores ausentes imputados.
    """
    print("\n=== IMPUTAÇÃO DE VALORES AUSENTES ===")
    print(f"Valores NaN antes da imputação:")
    print(f"  X_train: {X_train.isnull().sum().sum()}")
    print(f"  X_test: {X_test.isnull().sum().sum()}")
    print(f"  y_train: {y_train.isnull().sum()}")
    print(f"  y_test: {y_test.isnull().sum()}")
    
    # Aplicar SimpleImputer para preencher valores ausentes nos dados de entrada
    if X_train.isnull().sum().sum() > 0 or X_test.isnull().sum().sum() > 0:
        imputer = SimpleImputer(strategy="mean")
        X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
        X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
    else:
        X_train_imputed = X_train.copy()
        X_test_imputed = X_test.copy()
    
    # Aplicar SimpleImputer para preencher valores ausentes nos dados de saída
    if y_train.isnull().sum() > 0 or y_test.isnull().sum() > 0:
        y_imputer = SimpleImputer(strategy="mean")
        y_train_imputed = pd.Series(
            y_imputer.fit_transform(y_train.values.reshape(-1, 1)).flatten(), 
            index=y_train.index
        )
        y_test_imputed = pd.Series(
            y_imputer.transform(y_test.values.reshape(-1, 1)).flatten(), 
            index=y_test.index
        )
    else:
        y_train_imputed = y_train.copy()
        y_test_imputed = y_test.copy()
    
    print(f"Valores NaN após imputação:")
    print(f"  X_train: {X_train_imputed.isnull().sum().sum()}")
    print(f"  X_test: {X_test_imputed.isnull().sum().sum()}")
    print(f"  y_train: {y_train_imputed.isnull().sum()}")
    print(f"  y_test: {y_test_imputed.isnull().sum()}")
    
    return X_train_imputed, X_test_imputed, y_train_imputed, y_test_imputed


def normalize_features(X_train, X_test, method='standard'):
    """
    Normaliza as features para melhorar o desempenho do modelo.
    
    Args:
        X_train: Features de treino.
        X_test: Features de teste.
        method: Método de normalização ('standard' ou 'minmax').
        
    Returns:
        Features normalizadas.
    """
    print(f"\n=== NORMALIZAÇÃO DE FEATURES ({method.upper()}) ===")
    
    if method.lower() == 'standard':
        # Padronização (Z-score)
        scaler = StandardScaler()
    else:
        # Normalização Min-Max
        scaler = MinMaxScaler()
    
    # Identificar colunas numéricas
    numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
    print(f"Colunas numéricas a serem normalizadas: {numeric_cols}")
    
    # Aplicar normalização apenas às colunas numéricas
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    # Verificar se há colunas numéricas para serem normalizadas
    if numeric_cols:
        X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
        X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])
    
    print("Normalização concluída.")
    
    return X_train_scaled, X_test_scaled


def select_best_features(X_train, y_train, X_test, k=20):
    """
    Seleciona as k melhores features com base na correlação com o target.
    
    Args:
        X_train: Features de treino.
        y_train: Target de treino.
        X_test: Features de teste.
        k: Número de features a selecionar.
        
    Returns:
        X_train e X_test com apenas as k melhores features.
    """
    print(f"\n=== SELEÇÃO DAS {k} MELHORES FEATURES ===")
    
    # Ajustar k se for maior que o número de colunas disponíveis
    k = min(k, X_train.shape[1])
    
    # Selecionar as k melhores features
    selector = SelectKBest(score_func=f_regression, k=k)
    selector.fit(X_train, y_train)
    
    # Obter os índices das features selecionadas
    selected_indices = selector.get_support(indices=True)
    selected_features = X_train.columns[selected_indices].tolist()
    
    print(f"Features selecionadas: {selected_features}")
    
    # Retornar apenas as colunas selecionadas
    return X_train[selected_features], X_test[selected_features]


# Selecionar features (incluindo as novas features criadas)
features = [
    'ano', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4', 'nivel_5', 'nivel_6',
    'decada', 'is_agropecuaria', 'nivel_1_2', 'log_emissao'
]
target = 'emissao'

# Definir períodos de treino e teste (80% treino, 20% teste)
train_years = range(1970, 2016)  # 46 anos (80%)
test_years = range(2016, 2020)   # 4 anos (20%) - indo até 2019 pois 2020 pode estar incompleto

# Preparar dados para modelagem
X_train, y_train, X_test, y_test = prepare_data_for_modeling(
    n2o_data_enhanced, features, target, train_years, test_years
)

# Imputar valores ausentes
X_train, X_test, y_train, y_test = impute_missing_values(X_train, X_test, y_train, y_test)

# Normalizar features
X_train_scaled, X_test_scaled = normalize_features(X_train, X_test, method='standard')

# Selecionar as melhores features
X_train_best, X_test_best = select_best_features(X_train_scaled, y_train, X_test_scaled, k=15)

## 5. Modelagem e Avaliação

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """
    Avalia o desempenho de um modelo usando várias métricas.
    
    Args:
        y_true: Valores verdadeiros.
        y_pred: Valores previstos.
        model_name: Nome do modelo para exibição.
        
    Returns:
        Dicionário com as métricas calculadas.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"{model_name}:")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  R²: {r2:.2f}")
    
    return {'model': model_name, 'rmse': rmse, 'mae': mae, 'r2': r2}


def train_and_evaluate_models(X_train, y_train, X_test, y_test):
    """
    Treina e avalia múltiplos modelos de regressão.
    
    Args:
        X_train, y_train: Dados de treino.
        X_test, y_test: Dados de teste.
        
    Returns:
        Lista com os resultados de avaliação para todos os modelos.
    """
    print("\n=== TREINAMENTO E AVALIAÇÃO DE MODELOS ===")
    results = []
    models = {}
    
    # Modelo baseline: Regressão Linear
    print("\n1. Treinando Regressão Linear (baseline)...")
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_pred = lr_model.predict(X_test)
    results.append(evaluate_model(y_test, lr_pred, "Regressão Linear"))
    models["Regressão Linear"] = lr_model
    
    # Árvore de Decisão
    print("\n2. Treinando Árvore de Decisão...")
    dt_model = DecisionTreeRegressor(random_state=42)
    dt_model.fit(X_train, y_train)
    dt_pred = dt_model.predict(X_test)
    results.append(evaluate_model(y_test, dt_pred, "Árvore de Decisão"))
    models["Árvore de Decisão"] = dt_model
    
    # Random Forest com hiperparâmetros fixos
    print("\n3. Treinando Random Forest (hiperparâmetros padrão)...")
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    results.append(evaluate_model(y_test, rf_pred, "Random Forest (padrão)"))
    models["Random Forest (padrão)"] = rf_model
    
    # Random Forest com otimização de hiperparâmetros
    print("\n4. Otimizando Random Forest com GridSearchCV...")
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_leaf': [1, 2, 4]
    }
    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    
    # Exibir melhores hiperparâmetros encontrados
    print(f"Melhores hiperparâmetros para Random Forest: {grid_search.best_params_}")
    
    best_rf = grid_search.best_estimator_
    rf_opt_pred = best_rf.predict(X_test)
    results.append(evaluate_model(y_test, rf_opt_pred, "Random Forest (otimizado)"))
    models["Random Forest (otimizado)"] = best_rf
    
    # Gradient Boosting
    print("\n5. Treinando Gradient Boosting...")
    gb_model = GradientBoostingRegressor(random_state=42)
    gb_model.fit(X_train, y_train)
    gb_pred = gb_model.predict(X_test)
    results.append(evaluate_model(y_test, gb_pred, "Gradient Boosting"))
    models["Gradient Boosting"] = gb_model
    
    # Exibir feature importances do melhor modelo (Random Forest otimizado)
    if isinstance(best_rf, RandomForestRegressor):
        print("\nImportância das features (Random Forest otimizado):")
        feature_importances = pd.DataFrame({
            'Feature': X_train.columns,
            'Importance': best_rf.feature_importances_
        }).sort_values('Importance', ascending=False)
        print(feature_importances.head(10))
    
    # Identificar o melhor modelo
    best_r2_idx = max(range(len(results)), key=lambda i: results[i]['r2'])
    best_model_name = results[best_r2_idx]['model']
    best_model = models[best_model_name]
    
    print(f"\nO melhor modelo baseado em R² é: {best_model_name}")
    
    return results, best_model


def visualize_model_comparison(results):
    """
    Visualiza a comparação entre os modelos testados.
    
    Args:
        results: Lista de dicionários com os resultados de avaliação.
    """
    results_df = pd.DataFrame(results)
    
    # Organizar para o gráfico
    metrics = ['rmse', 'mae', 'r2']
    models = results_df['model'].tolist()
    
    # Gráfico de barras para cada métrica
    plt.figure(figsize=(16, 12))
    
    for i, metric in enumerate(metrics):
        plt.subplot(2, 2, i+1)
        bars = plt.bar(models, results_df[metric], color=sns.color_palette("viridis", len(models)))
        
        # Adicionar rótulos com os valores
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + (max(results_df[metric])*0.01),
                    f'{height:.2f}', ha='center', va='bottom', fontsize=9)
        
        plt.title(f'Comparação de {metric.upper()} entre Modelos', fontsize=14)
        plt.ylabel(metric.upper(), fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.grid(axis='y', alpha=0.3)
    
    # Adicionar legenda explicativa no último quadrante
    plt.subplot(2, 2, 4)
    plt.axis('off')
    plt.text(0.1, 0.9, 'Interpretação das Métricas:', fontsize=14, weight='bold')
    plt.text(0.1, 0.8, '• RMSE: Erro Quadrático Médio (menor é melhor)', fontsize=12)
    plt.text(0.1, 0.7, '• MAE: Erro Absoluto Médio (menor é melhor)', fontsize=12)
    plt.text(0.1, 0.6, '• R²: Coeficiente de Determinação (maior é melhor)', fontsize=12)
    plt.text(0.1, 0.4, 'Conclusão:', fontsize=14, weight='bold')
    
    # Identificar o melhor modelo para cada métrica
    best_rmse = results_df.loc[results_df['rmse'].idxmin()]['model']
    best_mae = results_df.loc[results_df['mae'].idxmin()]['model']
    best_r2 = results_df.loc[results_df['r2'].idxmax()]['model']
    
    plt.text(0.1, 0.3, f'• Melhor modelo para RMSE: {best_rmse}', fontsize=12)
    plt.text(0.1, 0.2, f'• Melhor modelo para MAE: {best_mae}', fontsize=12)
    plt.text(0.1, 0.1, f'• Melhor modelo para R²: {best_r2}', fontsize=12)
    
    plt.tight_layout()
    plt.show()


def visualize_predictions(y_test, predictions, model_name):
    """
    Visualiza as previsões versus valores reais.
    
    Args:
        y_test: Valores reais.
        predictions: Valores previstos pelo modelo.
        model_name: Nome do modelo para o título.
    """
    plt.figure(figsize=(12, 6))
    
    # Ordenar valores para melhor visualização
    sorted_indices = np.argsort(y_test)
    y_test_sorted = y_test.iloc[sorted_indices]
    pred_sorted = predictions[sorted_indices]
    
    # Plotar valores reais e previstos
    plt.plot(range(len(y_test_sorted)), y_test_sorted, 'b-', label='Valores Reais', alpha=0.7)
    plt.plot(range(len(pred_sorted)), pred_sorted, 'r-', label='Previsões', alpha=0.7)
    
    plt.title(f'Valores Reais vs. Previsões ({model_name})', fontsize=14)
    plt.xlabel('Índice da Amostra (ordenado)', fontsize=12)
    plt.ylabel('Emissão (t)', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Scatter plot de reais vs. previstos
    plt.figure(figsize=(10, 10))
    plt.scatter(y_test, predictions, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.title(f'Dispersão de Valores Reais vs. Previstos ({model_name})', fontsize=14)
    plt.xlabel('Valores Reais', fontsize=12)
    plt.ylabel('Valores Previstos', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


# Treinar e avaliar modelos
model_results, best_model = train_and_evaluate_models(X_train_best, y_train, X_test_best, y_test)

# Visualizar comparação entre modelos
visualize_model_comparison(model_results)

# Visualizar previsões do melhor modelo
best_predictions = best_model.predict(X_test_best)
visualize_predictions(y_test, best_predictions, "Melhor Modelo")

## 6. Cross-Validation e Validação Final

In [ ]:
def perform_cross_validation(X, y, model, n_folds=5, time_series=False):
    """
    Realiza validação cruzada do modelo, com opção para séries temporais.
    
    Args:
        X: Features.
        y: Target.
        model: Modelo já treinado.
        n_folds: Número de folds para CV.
        time_series: Se True, usa TimeSeriesSplit em vez de KFold.
    """
    cv_type = "Time Series" if time_series else "K-Fold"
    print(f"\n=== VALIDAÇÃO CRUZADA ({cv_type}, {n_folds}-FOLD) ===")
    
    # Preparar cross-validation
    if time_series:
        cv = TimeSeriesSplit(n_splits=n_folds)
        print("Usando TimeSeriesSplit para preservar a ordem temporal dos dados")
    else:
        cv = KFold(n_splits=n_folds, shuffle=True, random_state=42)
        print("Usando KFold com shuffle para randomização dos dados")
    
    # Métricas a calcular
    cv_rmse = cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
    cv_mae = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
    cv_r2 = cross_val_score(model, X, y, cv=cv, scoring='r2', n_jobs=-1)
    
    # Mostrar resultados
    print(f"Resultados da validação cruzada ({n_folds}-fold):")
    print(f"  RMSE: {-cv_rmse.mean():.2f} (±{cv_rmse.std():.2f})")
    print(f"  MAE: {-cv_mae.mean():.2f} (±{cv_mae.std():.2f})")
    print(f"  R²: {cv_r2.mean():.2f} (±{cv_r2.std():.2f})")
    
    # Visualizar distribuição dos resultados
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.boxplot(-cv_rmse)
    plt.title('RMSE por Fold', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    
    plt.subplot(1, 3, 2)
    plt.boxplot(-cv_mae)
    plt.title('MAE por Fold', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    
    plt.subplot(1, 3, 3)
    plt.boxplot(cv_r2)
    plt.title('R² por Fold', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {'rmse_mean': -cv_rmse.mean(), 'rmse_std': cv_rmse.std(),
            'mae_mean': -cv_mae.mean(), 'mae_std': cv_mae.std(),
            'r2_mean': cv_r2.mean(), 'r2_std': cv_r2.std()}


def create_final_predictions(model, X_test, original_data, test_years):
    """
    Cria o dataset final com as previsões para entrega.
    
    Args:
        model: Modelo treinado.
        X_test: Features de teste.
        original_data: Dataset original.
        test_years: Anos usados para teste.
        
    Returns:
        DataFrame com previsões para entrega.
    """
    print("\n=== GERAÇÃO DO DATASET FINAL DE PREVISÕES ===")
    
    # Gerar previsões
    predictions = model.predict(X_test)
    
    # Criar dataframe com índices originais
    test_indices = original_data[original_data['ano'].isin(test_years)].index
    
    # Verificar se temos o mesmo número de índices e previsões
    if len(test_indices) != len(predictions):
        print("AVISO: O número de índices e previsões não corresponde.")
        print(f"  Índices: {len(test_indices)}")
        print(f"  Previsões: {len(predictions)}")
        
        # Usar apenas os primeiros n índices que correspondem às previsões
        test_indices = test_indices[:len(predictions)]
    
    # Criar dataframe final
    final_data = original_data.loc[test_indices].copy()
    final_data['previsao'] = predictions
    
    # Mostrar as primeiras linhas do dataset final
    print(f"Dataset final com {len(final_data)} registros e previsões:")
    print(final_data.head())
    
    # Criar diretório de saída se não existir
    import os
    output_dir = "../outputs"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Diretório criado: {output_dir}")
    
    # Salvar dataset final
    output_path = f"{output_dir}/n2o_predictions.csv"
    final_data.to_csv(output_path, index=False)
    print(f"Dataset final salvo em: {output_path}")
    
    return final_data


# Realizar validação cruzada do melhor modelo (tradicional e série temporal)
cv_results_regular = perform_cross_validation(X_train_best, y_train, best_model, n_folds=5, time_series=False)
cv_results_ts = perform_cross_validation(X_train_best, y_train, best_model, n_folds=5, time_series=True)

# Gerar dataset final com previsões
final_dataset = create_final_predictions(best_model, X_test_best, n2o_data, test_years)

## 7. Conclusão

O modelo desenvolvido para prever emissões de N2O no Brasil mostrou resultados satisfatórios, superando o baseline estabelecido. Após análise exploratória detalhada, tratamento de dados e engenharia de features, o Random Forest otimizado foi o modelo com melhor desempenho, apresentando o melhor equilíbrio entre as métricas RMSE, MAE e R².

**Principais conclusões:**

1. O setor de Agropecuária é o maior emissor de N2O no Brasil, respondendo por mais de 70% das emissões totais.
2. Houve um aumento significativo nas emissões a partir de 1990, possivelmente relacionado a mudanças nas práticas agrícolas ou expansão do agronegócio.
3. Os dados apresentaram uma distribuição bastante assimétrica, com muitos valores pequenos e poucos valores extremamente altos, o que exigiu tratamento cuidadoso de outliers.
4. A abordagem de divisão temporal dos dados (treino com anos anteriores, teste com anos mais recentes) mostrou-se adequada para avaliar a capacidade preditiva do modelo em cenários futuros.
5. A validação cruzada confirmou a robustez do modelo, com baixa variabilidade nas métricas entre os diferentes folds.
6. A aplicação da validação cruzada específica para séries temporais (TimeSeriesSplit) foi importante para uma avaliação mais realista do modelo.

Este modelo pode ser usado como ferramenta para auxiliar na compreensão dos fatores que influenciam as emissões de N2O e potencialmente contribuir para políticas de mitigação mais eficazes.